In [1]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd

In [2]:
ns = '{http://www.volby.cz/prezident/}'

In [ ]:
data = pd.DataFrame()
for i in range(1, 54):
    r = requests.get('http://80.188.53.60/pls/prezmedia/vysledky_okrsky?kolo=&davka=' + str(i))
    root = ET.fromstring(r.text)
    for okr in root.findall(ns + 'OKRSEK'):
        tmp = okr.attrib
        tmp.update(okr.find(ns + 'UCAST_OKRSEK').attrib)
        for kand in okr.findall(ns + 'HLASY_OKRSEK'):
            tmp.update({'HLASY_0' + kand.get('PORADOVE_CISLO'): int(kand.get('HLASY'))})
        data = data.append(tmp, ignore_index=True)

In [4]:
data.columns

Index(['CIS_OBEC', 'CIS_OKRSEK', 'DATUM_CAS_ZPRAC', 'HLASY_02', 'HLASY_03',
       'HLASY_05', 'HLASY_07', 'HLASY_08', 'HLASY_09', 'ODEVZDANE_OBALKY',
       'OPAKOVANE', 'PLATNE_HLASY', 'PORADI_ZPRAC', 'VYDANE_OBALKY',
       'ZAPSANI_VOLICI', 'HLASY_01', 'HLASY_04', 'HLASY_06'],
      dtype='object')

In [5]:
data.rename(columns={
    'CIS_OBEC': 'OBEC', 
    'CIS_OKRSEK': 'OKRSEK', 
    'ODEVZDANE_OBALKY': 'ODEVZ_OBAL',
    'PLATNE_HLASY': 'PL_HL_CELK', 
    'VYDANE_OBALKY': 'VYD_OBALKY',
    'ZAPSANI_VOLICI': 'VOL_SEZNAM'
}, inplace=True)

In [ ]:
data.fillna(0, inplace=True)

In [ ]:
for col in data.columns:
    if col.startswith('HLASY_'):
        data[col] = data[col].astype('int')

In [10]:
data.to_csv('./okrsky_temp.csv', encoding='utf-8', index=False)